# Wordle Agent


WORDLE GAME:


![WordleGame](https://media.phillyvoice.com/media/images/011322_Wordle.2e16d0ba.fill-735x490.png)


### Game Rules: - You have **6 tries** to guess a secret **5-letter** word. 
- Each guess must be a valid **5-letter English word**. 
- After each guess, you will receive feedback indicating how close your guess was. 

### Feedback Format: Each letter in your guess will receive one of three vlaues: 
- G (green): correct letter in the correct position
- Y (yellow): letter exists in the word but in the wrong position
- X (wrong): letter is not in the word


In [ ]:
from datasets import load_dataset
import pandas as pd
import dspy
from dotenv import load_dotenv
from pprint import pprint
from typing import Literal, Union
load_dotenv()

gpt4o_mini = dspy.LM('gpt-4o-mini')
dataset = load_dataset("willcb/V3-wordle", split="train").to_pandas()


/gaueko0/users/asalem/anaconda3/envs/py310_transformers4dot49/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset.head(3)

In [ ]:
class GuessedWordFeedback(dspy.Signature):

    guessed_word: str = dspy.InputField(desc="The predicted word by the player")
    actual_word: str = dspy.InputField(desc="The actual word to be guessed")
    feedback: str = dspy.OutputField(
        desc="""Feedback Format: Each letter in your guess will receive one of three values: 
- G (green): correct letter in the correct position
- Y (yellow): letter exists in the word but in the wrong position
- X (wrong): letter is not in the word"""
    )

## Test the signature
with dspy.context(lm=gpt4o_mini):
    feedback_player = dspy.ChainOfThought(GuessedWordFeedback)
    response = feedback_player(guessed_word="apple", actual_word="plane")
    pprint(response)

In [ ]:
class WordleGuess(dspy.Signature):

    feedback: str = dspy.InputField(
        desc="""Feedback from the previous guess, if any. Feedback Format: Each letter in your guess will receive one of three values: 
- G (green): correct letter in the correct position
- Y (yellow): letter exists in the word but in the wrong position
- X (wrong): letter is not in the word. First guess has no feedback."""
    )

    guessed_word: str = dspy.OutputField(
        desc="The guessed word based on the feedback from the previous guess. only alphabetic characters are allowed, and it must be a valid 5-letter English word.",
    )

## Test the signature

with dspy.context(lm=gpt4o_mini):
    wordle_guess = dspy.ChainOfThought(WordleGuess)

    # Example usage
    response = wordle_guess(feedback="First Guess, no feedback")
    pprint(response)

In [ ]:
class Wordle(dspy.Module):
    """The game of Wordle.
    Game Rules: - You have **6 tries** to guess a secret **5-letter** word.
    - Each guess must be a valid **5-letter English word**.
    - After each guess, you will receive feedback indicating how close your guess was.

    ### Feedback Format: Each letter in your guess will receive one of three values:
    - G (green): correct letter in the correct position
    - Y (yellow): letter exists in the word but in the wrong position
    - X (wrong): letter is not in the word."""

    def __init__(
        self, player1_lm, player2_lm, max_attempts=6, is_training=False, verbose=False, score_function=None
    ):
        super().__init__()
        self.player1 = dspy.ChainOfThought(WordleGuess)
        self.player1.set_lm(player1_lm)
        self.player2 = dspy.ChainOfThought(GuessedWordFeedback)
        self.player2.set_lm(player2_lm)
        self.max_attempts = max_attempts
        self.training_mode = is_training
        self.verbose = verbose
        
        if is_training:
            assert score_function is not None, "Score function must be provided in training mode."
            self.score_function = score_function

    def forward(self, word):

        # first attempt has no feedback
        feedback = "Guess the word!"

        for attempt in range(self.max_attempts):

            guess = self.player1(feedback=feedback)
            if guess.guessed_word == word:
                # print(f"Predicted Word: {guess.guessed_word}, Feedback: Correct!")
                return (
                    guess.guessed_word
                    if self.training_mode
                    else "Player 1 Wins after {} attempts!".format(attempt + 1)
                )

            feedback = self.player2(guessed_word=guess.guessed_word, actual_word=word)
            if self.verbose:
                pprint(
                    f"Attempt {attempt + 1}: Predicted Word: {guess.guessed_word}, Feedback: {feedback.feedback}"
                )
                
        
        return guess.guessed_word if self.training_mode else "Sorry, you've run out of attempts."

In [ ]:
wordle_game = Wordle(player1_lm=dspy.LM('gpt-4o-mini'), player2_lm=dspy.LM('gpt-4o-mini'), max_attempts=6, is_training=False, verbose=True)

In [ ]:
wordle_game(word="clock")

In [ ]:
wordle_game(word="plane")

In [ ]:
wordle_game(word="stick")


# Optimize the Prompt Automatically with MiPROv2

Multiprompt Instruction PRoposal Optimizer Version 2

## Define our Dataset

In [ ]:
from dspy.datasets.dataset import Dataset

class WordleDataser(Dataset):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        ds = load_dataset("willcb/V3-wordle", split="train").remove_columns(['prompt', 'completion', 'reward', 'task']).train_test_split(test_size=0.25)
        train_ds = ds['train'].to_pandas()
        test_ds = ds['test'].to_pandas()
        
        self._train = train_ds.to_dict(orient='records')
        self._dev = test_ds.to_dict(orient='records')


wordle_dataset = WordleDataser()
pprint(wordle_dataset.train[:3])

## Define a Metric!
Non Differentiable 


In [ ]:
from dspy.evaluate import answer_exact_match

from dspy.data